In [1]:
import pandas as pd
import altair as alt
import eco_style
alt.themes.enable("light")

ThemeRegistry.enable('light')

# Ipsos: More harm then good

In [53]:
import altair as alt
import pandas as pd

# Read and reshape your data
df = pd.read_csv("data/ipsos_Increasing_tariffs_on_imported_goods_will_do_more_harm_than_good.csv")
df = df.replace("%", "", regex=True)
df = df.melt(id_vars=["Response"], var_name="group", value_name="value")
df.group = df.group.str.replace("\(.*\)", "", regex=True)
df['value'] = df['value'].astype(float) / 100

# Filter only relevant responses
df = df[df.Response.isin(["Strongly agree", "Somewhat agree", "Unsure", "Somewhat disagree", "Strongly disagree",])]

# Assign direction: positive (right) or negative (left) for centering
df["direction"] = df["Response"].map({
    "Strongly agree": 1,
    "Somewhat agree": 1,
    "Unsure": 1,
    "Somewhat disagree": -1,
    "Strongly disagree": -1
})
df["adjusted_value"] = df["value"] * df["direction"]

# Order for stacking
df["response_order"] = df["Response"].map({
    "Strongly disagree": 0,
    "Somewhat disagree": 1,
    "Unsure": 0,
    "Somewhat agree": 3,
    "Strongly agree": 4
})

# Colour map
color_scale = alt.Scale(
    domain=[
        "Strongly agree", "Somewhat agree",
        "Unsure",
        "Somewhat disagree", "Strongly disagree"
    ],
    range=[
        "#36B7B4", "#A0DDDB",
        "#C9C9C9",
        "#F58A9D", "#E6224B"
    ]
)


# Sorting group order
order = ["Total", "Democrat", "Independent", "Republican"]
order = ["Republican", "Independent", "Democrat", "Total"]
df['group'] = df['group'].str.strip()
df = df[df["group"].isin(order)]
df["group"] = pd.Categorical(df["group"], categories=order[::-1], ordered=True)

# Create an order column for sorting in Altair
group_order = ["Total", "Democrat", "Independent", "Republican"]
order_map = {name: i for i, name in enumerate(group_order[::-1])}  # Reverse order (Republican at top)
df["group_order"] = df["group"].map(order_map)


# Chart
chart = alt.Chart(df).mark_bar().encode(
    y=alt.Y("group:N", title=None,
            sort=alt.EncodingSortField(field="group_order", order="ascending")),
    x=alt.X("adjusted_value:Q", axis=alt.Axis(format="%"), title=None),
    color=alt.Color("Response:N", scale=color_scale, legend=alt.Legend(title="", orient="top", values=["Strongly disagree", "Somewhat disagree", "Unsure", "Somewhat agree", "Strongly agree"])),
    order=alt.Order("response_order:O", sort="descending"),
)
chart

/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

## Non-centered

In [101]:
import altair as alt
import pandas as pd

# Read and reshape your data
df = pd.read_csv("data/ipsos_Increasing_tariffs_on_imported_goods_will_do_more_harm_than_good.csv")
df = df.replace("%", "", regex=True)
df = df.melt(id_vars=["Response"], var_name="group", value_name="value")
df.group = df.group.str.replace("\(.*\)", "", regex=True)
df['value'] = df['value'].astype(float) / 100


# Keep only the main responses
df = df[df.Response.isin(["Strongly agree", "Somewhat agree", "Unsure", "Somewhat disagree", "Strongly disagree"])]

df["value"] = df["value"] / df.groupby("group")["value"].transform("sum")

# Define stacking order
df["response_order"] = df["Response"].map({
    "Strongly disagree": 0,
    "Somewhat disagree": 1,
    "Unsure": 2,
    "Somewhat agree": 3,
    "Strongly agree": 4
})

# Set colours
color_scale = alt.Scale(
    domain=[
        "Strongly disagree", "Somewhat disagree",
        "Unsure",
        "Somewhat agree", "Strongly agree"
    ],
    range=[
        "#E6224B", "#F58A9D",
        "#C9C9C9",
        "#A0DDDB", "#36B7B4"
    ]
)

# Sorting group order
order = ["Total", "Democrat", "Independent", "Republican"]
df['group'] = df['group'].str.strip()
df = df[df["group"].isin(order)]
group_order = order[::-1]  # Republican at top
df["group_order"] = df["group"].map({name: i for i, name in enumerate(group_order)})

# Plot
chart = alt.Chart(df).mark_bar().encode(
    y=alt.Y("group:N", sort=alt.EncodingSortField(field="group_order", order="descending"), title=None),
    x=alt.X("value:Q", title=None, axis=alt.Axis(format="%")),
    color=alt.Color("Response:N", scale=color_scale, legend=alt.Legend(title="", orient="top")),
    order=alt.Order("response_order:O", sort="ascending")
)

chart.save("charts/US_tarrifs_will_do_more_harm_than_good.png", scale_factor=3)
chart.save("charts/US_tarrifs_will_do_more_harm_than_good.json")
chart

/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=Fa

alt.Chart(...)

## Partisan Version

In [115]:
import altair as alt
import pandas as pd

# Read and reshape your data
df = pd.read_csv("data/ipsos_Increasing_tariffs_on_imported_goods_will_do_more_harm_than_good.csv")
df = df.replace("%", "", regex=True)
df = df.melt(id_vars=["Response"], var_name="group", value_name="value")
df.group = df.group.str.replace("\(.*\)", "", regex=True)
df['value'] = df['value'].astype(float) / 100


# Keep only the main responses
df = df[df.Response.isin(["Strongly agree", "Somewhat agree", "Unsure", "Somewhat disagree", "Strongly disagree"])]


df["value"] = df["value"] / df.groupby("group")["value"].transform("sum")

# Define stacking order
df["response_order"] = df["Response"].map({
    "Strongly disagree": 0,
    "Somewhat disagree": 1,
    "Unsure": 2,
    "Somewhat agree": 3,
    "Strongly agree": 4
})

# Set colours
color_scale = alt.Scale(
    domain=[
        "Strongly disagree", "Somewhat disagree",
        "Unsure",
        "Somewhat agree", "Strongly agree"
    ],
    range=[
        "#E6224B", "#F58A9D",
        "#C9C9C9",
        "#A0DDDB", "#36B7B4"
    ]
)

# Keep just republicans and democrats

# Sorting group order
order = ["Total", "Democrat", "Independent", "Republican"]
df['group'] = df['group'].str.strip()
df = df[df["group"].isin(order)]
group_order = order[::-1]  # Republican at top
df["group_order"] = df["group"].map({name: i for i, name in enumerate(group_order)})

df = df[df["group"].isin(["Democrat", "Republican"])]


# Plot
chart = alt.Chart(df).mark_bar().encode(
    y=alt.Y("group:N", sort=alt.EncodingSortField(field="group_order", order="descending"), title=None),
    x=alt.X("value:Q", title=None, axis=alt.Axis(format="%")),
    color=alt.Color("Response:N", scale=color_scale, legend=alt.Legend(title="", orient="top")),
    order=alt.Order("response_order:O", sort="ascending")
).properties(
    width=475,
    height=200
)

chart.save("charts/US_tarrifs_will_do_more_harm_than_good_dem_rep.png", scale_factor=3)
chart.save("charts/US_tarrifs_will_do_more_harm_than_good_dem_rep.json")
chart

/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=Fa

alt.Chart(...)

In [118]:
df.query("group == 'Republican'")

,Response,group,value,response_order,group_order
16,Strongly agree,Republican,0.08,4,0
17,Somewhat agree,Republican,0.13,3,0
18,Somewhat disagree,Republican,0.32,1,0
19,Strongly disagree,Republican,0.27,0,0
20,Unsure,Republican,0.20,2,0


In [119]:
df

,Response,group,value,response_order,group_order
8,Strongly agree,Democrat,0.673267,4,2
9,Somewhat agree,Democrat,0.188119,3,2
10,Somewhat disagree,Democrat,0.039604,1,2
11,Strongly disagree,Democrat,0.039604,0,2
12,Unsure,Democrat,0.059406,2,2
16,Strongly agree,Republican,0.080000,4,0
17,Somewhat agree,Republican,0.130000,3,0
18,Somewhat disagree,Republican,0.320000,1,0
19,Strongly disagree,Republican,0.270000,0,0
20,Unsure,Republican,0.200000,2,0


In [108]:
df

,Response,group,value,response_order,group_order


# Chart: Retaliation

In [88]:
df

,Country,Response,value,response_order,country_order
1,France,Strongly support,0.4163,4,3
2,Germany,Strongly support,0.3700,4,1
3,Denmark,Strongly support,0.5100,4,6
4,Sweden,Strongly support,0.4600,4,5
5,Spain,Strongly support,0.4300,4,4
6,Italy,Strongly support,0.3100,4,0
8,France,Somewhat support,0.2559,3,3
9,Germany,Somewhat support,0.3100,3,1
10,Denmark,Somewhat support,0.2800,3,6
11,Sweden,Somewhat support,0.2700,3,5


In [120]:
df = pd.read_csv("https://interactive.guim.co.uk/datawrapper/embed/SU8Bo/4/dataset.csv", sep="\t")
df.to_csv("data/yougov_guardian_Do you support or oppose retaliatory tariffs on US goods.csv", index=False)

# Rename GB to 'Great Britain' for clarity
df["Country"] = df["Country"].replace({"Britain": "Great Britain"})

# Melt the dataframe to long format
df = df.melt(id_vars=["Country"], var_name="Response", value_name="value")

# Convert to proportion
df["value"] = df["value"].astype(float) / 100
df["value"] = df["value"] / df.groupby("Country")["value"].transform("sum")


# Define response stacking order
response_order_map = {
    "Strongly oppose": 0,
    "Somewhat oppose": 1,
    "Don't know": 2,
    "Somewhat support": 3,
    "Strongly support": 4
}
df["response_order"] = df["Response"].map(response_order_map)

# Colour scale
color_scale = alt.Scale(
    domain=list(response_order_map.keys()),
    range=["#E6224B", "#F58A9D", "#C9C9C9", "#A0DDDB", "#36B7B4"]
)

# Set country order
country_order = ['Spain', 'Italy', 'France', 'Great Britain', 'Germany', 'Sweden', 'Denmark']
country_order = ["Denmark", "Sweden", "Spain", "France", "Great Britain", "Germany", "Italy"]
df = df[df["Country"].isin(country_order)]
df["country_order"] = df["Country"].map({name: i for i, name in enumerate(reversed(country_order))})

# Build chart
chart = alt.Chart(df).mark_bar().encode(
    y=alt.Y("Country:N", sort=alt.EncodingSortField(field="country_order", order="descending"), title=None),
    x=alt.X("value:Q", axis=alt.Axis(format='%'), title=None),
    color=alt.Color("Response:N", scale=color_scale, legend=alt.Legend(title="", orient="top", values=list(response_order_map.keys().__reversed__()
                                                                                                           ))),
    order=alt.Order("response_order:O", sort="descending"),
).properties(
    width=420,
    height=250,
)

halfway_rule = alt.Chart(pd.DataFrame({"x": [0.5]})).mark_rule(
    strokeDash=[3, 5],
    color="#676A86", strokeWidth=1).encode(
    x=alt.X("x:Q", title=None, axis=alt.Axis(format="%"))
)

chart = chart + halfway_rule
chart.save("charts/yougov_guardian_Do you support or oppose retaliatory tariffs on US goods.png", scale_factor=3)
chart.save("charts/yougov_guardian_Do you support or oppose retaliatory tariffs on US goods.json")
chart


/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=Fa

alt.LayerChart(...)

In [124]:
df.query("Country == 'Denmark'")

,Country,Response,value,response_order,country_order
3,Denmark,Strongly support,0.51,4,6
10,Denmark,Somewhat support,0.28,3,6
17,Denmark,Don't know,0.12,2,6
24,Denmark,Somewhat oppose,0.06,1,6
31,Denmark,Strongly oppose,0.03,0,6


In [123]:
0.41+0.3-0.07-0.04

0.5999999999999999

/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [48]:
df.Response.unique()

array(['Strongly agree', 'Somewhat agree', 'Somewhat disagree',
       'Strongly disagree', 'Unsure'], dtype=object)

In [21]:
df

,Response,group,value,direction,adjusted_value,response_order
0,Strongly agree,Total,0.36,1,0.36,4
1,Somewhat agree,Total,0.16,1,0.16,3
2,Somewhat disagree,Total,0.17,-1,-0.17,1
3,Strongly disagree,Total,0.13,-1,-0.13,0
4,Unsure,Total,0.16,0,0.00,2
8,Strongly agree,NaN,0.68,1,0.68,4
9,Somewhat agree,NaN,0.19,1,0.19,3
10,Somewhat disagree,NaN,0.04,-1,-0.04,1
11,Strongly disagree,NaN,0.04,-1,-0.04,0
12,Unsure,NaN,0.06,0,0.00,2
